In [1]:
import os
from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq
from typing import TypedDict
from dotenv import load_dotenv


In [2]:
groq_api_key = os.getenv("GROQ_API_KEY")

model = ChatGroq(model ="Gemma2-9b-It",groq_api_key=groq_api_key)
model.invoke("HI GROQ, tell me who is David Goggins and list his prime achievements?")


AIMessage(content='David Goggins is an American ultramarathon runner, ultra-distance cyclist, triathlete, motivational speaker, and author. \n\nHe\'s known for his incredible physical and mental toughness, pushing himself beyond what most consider humanly possible. \n\nHere are some of his prime achievements:\n\n* **Navy SEAL:** Goggins served as a Navy SEAL, completing Hell Week, the notoriously grueling training program, despite being severely undersized and facing intense physical and psychological challenges.\n* **Ultra-Endurance Athlete:** He has broken numerous world records in ultra-endurance events, including:\n    * Running 100 miles in under 15 hours\n    * Completing 4,030 pull-ups in 48 hours\n    * Running 143 miles in 24 hours while carrying a 40-pound backpack\n* **Author:** Goggins is the author of the bestselling book "Can\'t Hurt Me," which details his incredible journey from childhood adversity to becoming a top athlete and motivational speaker.\n* **Motivational Spe

In [3]:
# create a state
class LLMState(TypedDict):
    question : str
    answer : str


In [4]:
def llm_qa(state: LLMState) -> LLMState:
    #extract the question from the state
    question = state['question']

    #form a prompt
    prompt =f'Answer the Following question {question}'

    # ask that question to the LLM
    response = model.invoke(prompt).content

    # UPDATE THE answer in the state
    state['answer'] = response

    return state

In [5]:
# create out graph
graph = StateGraph(LLMState)

#add nodes
graph.add_node('llm_qa',llm_qa)

#add edges
graph.add_edge(START,'llm_qa')
graph.add_edge('llm_qa',END)

#compile
workflow = graph.compile()

In [6]:
#execute
initial_state ={'question':"Explain Prompt Chaining in Langgraph in 50 Words"}
final_state= workflow.invoke(initial_state)
final_state

{'question': 'Explain Prompt Chaining in Langgraph in 50 Words',
 'answer': "Prompt chaining in LangChain allows you to break down complex tasks into smaller, sequential prompts. \n\nEach prompt's output becomes the input for the next, enabling the model to perform multi-step reasoning and generate more sophisticated responses. \n\n"}

In [7]:
'''
Prompt Chaining: 

Prompt chaining in Langgraph leverages the outputs of previous prompts as input for subsequent ones, creating a chain of reasoning and enabling complex tasks.
It's like a conversation where each response informs the next, guiding the language model towards a desired outcome.
'''

"\nPrompt Chaining: \n\nPrompt chaining in Langgraph leverages the outputs of previous prompts as input for subsequent ones, creating a chain of reasoning and enabling complex tasks.\nIt's like a conversation where each response informs the next, guiding the language model towards a desired outcome.\n"

In [9]:
prompt =f'Answer the Following question why did mahatma gandhi die?'

    # ask that question to the LLM
response = model.invoke(prompt)
response


AIMessage(content="Mahatma Gandhi was assassinated by **Nathuram Godse**, a Hindu nationalist, on **January 30, 1948**. \n\nGodse opposed Gandhi's philosophy of non-violence and his support for a separate Muslim-majority nation (Pakistan) during the partition of India. \n\nHe believed that Gandhi was too conciliatory towards Muslims and that his policies were weakening India.\n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 86, 'prompt_tokens': 22, 'total_tokens': 108, 'completion_time': 0.156363636, 'prompt_time': 0.001338058, 'queue_time': 0.254262061, 'total_time': 0.157701694}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--481ac414-be9d-4cbc-ab51-02fa0b8b4357-0', usage_metadata={'input_tokens': 22, 'output_tokens': 86, 'total_tokens': 108})